In [2]:
!pip install unidecode
from unidecode import unidecode
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

def csv_files(path):
    """Função usada para ler os ficheiros que tem o simbolo do Euro"""
    import csv
    f = open(path, encoding='windows-1252', errors='ignore')
    data = []
    for row in csv.reader(f, delimiter=','):
        data.append(row)
    col = [*data[0]]
    data.pop(0)
    df = pd.DataFrame(data, columns=col)
    return df, col

def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):
    """
    Params:\n
    path: parth of file in the computer.\n
    n_skiprows: Number of rows to delete in the original file,.\n
    columns_to_convert: Columns to convert the data general type. \n
    n_skipcolumn: Columns to skip in the original file. \n
    endrow = pass 0 to read everything, 1 to count entire
    columns_order: List of columns names in specific order to pass in the engine.\n
    """
    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    """cont = 0
    for i in df.iloc[:,site_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[:cont, :]
    #df.columns = columns_order"""
    
    # convert intery columns to integer 
    #df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
    #df[columns_integer_convert] = df[columns_integer_convert].astype('int64')

    return df

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def check_df(df, error_msg):
    if df == None or df.empty:
        return f'{error_msg}'
    else:
        return df

def count_duplicates(lista):
    count_dict = {}
    for entry in lista:
        if entry in count_dict.keys():
            count_dict[entry] += 1
        else:
            count_dict[entry] = 1
    
    duplicates = {}
    for k, v in count_dict.items():
        if v > 1:
            duplicates[k] = v
    return pd.DataFrame.from_dict(duplicates, orient='index', columns=['# of Duplicates'])

def defining_df(df, column_range, number_col):
    cont = 0
    for i in df.iloc[:,number_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[0:cont, :]
    return df

#old version
def check_columns(table, output_columns):
    """
    Check the total of number of missing columns and the missing columns in passed table.\n

    Params:\n
    table: contain the columns to be check.\n
    output_columns: columns structure at the final file. 

    Returns:\n
    Number of missing columns and a list that contains the name os missing columns.
    """
    """"
    countries = ['DE':{'towerDB':[],\
                       'lc': [],\
                       'ta':[]}\,
                 'HU',
                 'IE',
                 'RO',
                 'PT',
                 'ES',
                 'CZ': {'towerDB':["Code (Duplicate)","Site Status","VF - In scope / out of scope (Generalised scoping)","Site in Skylon scope Actual (From Site List Sheet )","Legacy Site Code(Duplicate)","TIMS Site Code","Legacy Site Code","Site Name","Macro Region","Province","Municipality","Inhabitants","Address","Ground Register","Altitude","Latitude","Longitude","Categorization by Inhabitants","Categorization by Transmission Sys","Categorization by Site Type","Categorization by Transmission Sys (subcluster)","Other internal Categorization 1 (Identify ACQ Sites)","Other internal Categorization 2 Energy provider (Eon/ LL)","DAS+Macro","DAS (Yes/ No)","DAS Ownership (Complete/ Partial/ 3rd Party)","Active/ Passive DAS","# of remote units/ radiating points","Type of Structure","Distance highest antenna to ground level","GBT Tower height","POD ID","Energy Consumption LTM (kwh)","Annual Energy cost LTM (Euros)","Infrastructure ready (existing)/ to be ready (new)","Infrastructure to be dismantled by","Radio equipments to be deactivated by","Infrastructure to be shared by","Technology VOD","Fibre / Microwave","Vertical passive structure owner","Room configuration (detailed)","Shelter passive structure ownership","Type of Air Conditioning","Number of cabinets (Full Capacity)","Number of Antenna (Full Capacity)","Number of MW (Full Capacity)","Counterpart","# of Lease Contracts","Current annual lease fees ","Current other fees (Maintenance)","Current other fees","(Average) residual duration - Lease contract","(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both","# of Tenants Agreements","Current Total Annual Hosting Fees","Tenant (name/ID) MNO1 (Česká telekomunikační infrastruktura a.s.)","Annual Fee per Tenant MNO1","Annual Energy Fee MNO1","Annual Maintenance Fee MNO1","Other Services Fee MNO1","Residual duration MNO1 (Years)","Tenant (name/ID) MNO2 (T-Mobile Czech Republic a.s.)","Annual Fee per Tenant MNO2","Annual Energy Fee MNO2","Annual Maintenance Fee MNO2","Other Services Fee MNO2","Residual duration MNO2 (days)","Tenant (name/ID) MNO3","Annual Fee per Tenant MNO3","Annual Energy Fee MNO3","Annual Maintenance Fee MNO3","Other Services Fee MNO3","Residual duration MNO3","# of OTMOs","Annual Fee from OTMOs","Annual Energy Fee from OTMOs","Annual Maintenance Fee OTMOs","Other Services Fee OTMOs","Average residual duration (days)","Check","Strategic Macro Sites","Critical Sites","Case A Core Site","Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters","Transmission Sites","Room Configuration","Power Supply","Air Conditioning","Active Sharing Arrangements involving the Operator","VF-CZ Demerger phase","EVO Location [FAR Site ID] ","Billing Trigger date ",\
                 "Strategic_Site_Bucket","Critical Site Bucket","Subsequent_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","Decommissioned_Site","Wip_Site","Bts_Site","Transfer_Date_Of_Consent_Required_Sites","Sites_As_Metered_Estimated","Date_Of_Equipment_Removal"],\
                       'lc': [],\
                       'ta':['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', 'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', 'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index','Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', 'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', 'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month','Updated Item', 'Comment', 'Counterpart_extra_1', 'Counterpart_extra_2', 'x','SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count', 'not_def_1']}\,
                 }
                 'GR']
                 """
    total_received = len(table.columns)
    number_missing_columns = 0
    missing_columns = []
    #Counting of missing columns       
    #if country in contries:      
    for columns in output_columns:
        if columns.lower() not in [labels.lower() for labels in table]:
            number_missing_columns +=1
            missing_columns.append(columns)
    
    return total_received, number_missing_columns, missing_columns

def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", 0, '-', '_', np.nan,'nan']
    #lista = []

    df.fillna(0, inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df
      
def date_parser(df, columns, format, type_dates='normal'):
    t_col = type_dates.lower()
    for column in columns:
        if t_col == 'mixed':
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)

# Refactorar esse codigo para receber todas as colunas num dic
# Sendo as keys=columns e values= picklist for each column
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')


def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')
        
def check_picklist(df,df_index,df_status, df_cols, picklist_dict):

    df_picklist = df[df_cols]
    df_picklist['sites'] = df[df_index]
    df_picklist =  df_picklist.set_index('sites')
    
    #df_picklist = replace_values(df_picklist, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        columns = [i.lower() for i in picklist_dict.keys()]
        for column in set(columns): 
            value = str(df_picklist.loc[site,column])
            if not value in set(picklist_dict[column]) or pd.isnull(value):
                #print(set(picklist_dict[column]))
                if not df_picklist.loc[site,df_index] in df_errors.index:
                    df_errors.loc[site,df_index] = df_picklist.loc[site,df_index]
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'
                else:
                    
                    if pd.isnull(value) or pd.isna(value) or value == 'nan' or value == '':
                        df_errors.loc[site,column] = 'Blank Value'
                    else:
                        df_errors.loc[site,column] = f'Incorret picklist value: {value}'

    #df = df_errors.dropna()   
    df_errors = df_errors.dropna(how='all', axis=1).fillna('Ok!')
    if len(df_errors)>0:
        df = df[[df_index, df_status]]
        df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
        df_errors = df_errors.set_index(df_index)
        df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
        df_errors = df_errors.reset_index()
    else:
        print('\nNo one Picklist Error Founded!')
    return df_errors

def check_picklist_3(df,df_index, picklist_dict):
    log = {}
    for column in picklist_dict:
        df_aux = df.copy()
        new = df_aux[column].isin(picklist_dict[column])
        #new = df_aux[column].apply(lambda x: x in picklist_dict[column])
        # Aceita somento os valores que não estão na picklist
        indexes = df.index[new == False].tolist()
        #if len(indexes)>0:
        if column not in log:log[column]=[]
        log[column]=log[column]+indexes
    #print(log.keys())

    newDict ={}
    df1 = pd.DataFrame()
    for key,value in log.items():
        for val in value:  
            ID=df.iloc[val][df_index]
            if ID in newDict:
                newDict[ID].append(key)
            else:
                newDict[ID] = [key]
        
    logs = pd.DataFrame.from_dict(newDict, orient='index')
    return logs

#check on air foi trocado pelo check_picklist
def check_on_air_sites(df, df_index, df_cols):

    df_check = df[df_cols]
    df_check['sites'] = df_check[df_index]
    df_check = df_check.set_index('sites')
    #df_check = replace_values(df_check, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        for column in df_cols: 
            if df_check.loc[site,df_index] not in df_errors.index:
                df_errors.loc[site,df_index] = df_check.loc[site,df_index]
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
            else:
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    df = df_errors[df_errors.iloc[:,1:]]
    df = df.dropna(how='all', axis=1).fillna('Ok')       
    #df = df_errors.dropna(how='all', axis=1)   
    return df


    """Restruturar o script em CZ, DE, PT"""

def check_bts(df_tw, bts_tw_columns, tw_index, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    return df

def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[wip_tw]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    return wip_out_tw_list, tw_wip_site_bts_flagged
    """Reestrurar os script em PT, DE, CZ"""
    # Falta os outros países

def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]
    """Ajustar para CZ, DE, PT"""
    
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
            return filtered[[tw_index, status_col, date_col]] 
        else:
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
            return filtered[[tw_index, status_col, date_col]] 
    else:
        print('Nothing wrong in services sites!')
    """Ajustar para CZ, DE, PT, IE"""

def check_tw_doer_planned(df_tw, tw_index, doer_col, status_col):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[doer_col].empty:
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)]
        return filtered[[tw_index, status_col, doer_col]]  
    else:
        print('Nothing wrong in services sites!')
                                                              
def check_mom_bts(df_tw, tw_index, tw_col, df_msa,msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    return filtered[[tw_index, tw_col]]         

def check_lc_ta_dates(df,tw_index, start_date,end_date):
    df[start_date] = pd.to_datetime(df[start_date])
    df[end_date] = pd.to_datetime(df[end_date], errors='coerce')
    filtered = df.loc[pd.to_datetime(df[start_date]) > df[end_date], [tw_index, start_date,end_date]]
    print(filtered)
    if not filtered.empty:
        return filtered[[tw_index, start_date,end_date]]
    else:
        print('\nNo Errors Founded!\n')

def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):
    t1 = ['pt', 'de', 'cz', 'ie', 'es', 'ro', 'hu']
    if country.lower() in t1:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        if decom_col != "":
            tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
            decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIP File'])
        
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIP File'])

        
            return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical
        else:
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIP File'])
            return in_service_uip_sites, bts_sites_out_uip, critical
    else:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index]]
        decomiss_sites_in_uip = []
        if set(tw_decomiss).intersection(uip_sites):
            decomiss_sites_in_uip = [i for i in tw_decomiss if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(decomiss_sites_in_uip, columns=['Decomissioned Site in UIP File'])
        
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

        #  Check for UIP critical sites 
        uip = [i for i in df_uip['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Yes'][tw_index]
        critical = []
        if set(uip).intersection(bts_tw_critical):
            critical = [i for i in bts_tw_critical if i not in uip]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% out in UIP File'])

        return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical

def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    return df_commercial_diffs

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 

def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, type_file='mix', status_col='', kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        if kind == 'ta':
            df['sites'] = df[index_col].astype(str) + df[kind_col]

            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = df_OLD.loc[row,col]
                    value_NEW = df_NEW.loc[row,col]
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old,  engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW,header=0, names=cols_new).fillna('')
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')


Compare oldest and newest files.

In [ ]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', 
                             status_col='', kind='tw', dates=[], sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col):
        #print(df.head(1))
        df = df.dropna(subset=[index_col], axis=0)
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = df_OLD.loc[row,col]
                    value_NEW = df_NEW.loc[row,col]
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    def date_parser(df, columns, format=1, type_dates='normal'):
        t_col = type_dates.lower()
        if format == 1:
            type_date = "%d/%m/%Y"
        else:
            type_date = "%d-%m-%Y"
        for column in lower_str(columns):
            if t_col == 'mixed':
                df[column] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW,header=0, names=cols_new).fillna('')
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,encoding='latin1').fillna('')
        cols_old = lower_str(list(df_OLD.columns))
        df_OLD.columns = cols_old
        df_OLD = fit_df(df_OLD, index_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        for i in dates:
            df_NEW[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) \
                         and not isinstance(date_obj, str) else date_obj for date_obj in df_NEW[i]]
            df_NEW[i] = df_NEW[i].astype(str)
            
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        df_NEW[index_col] = df_NEW[index_col].astype(str)
        df_NEW['sites'] = df_NEW[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_NEW = df_NEW.set_index('sites').fillna('')
        #df_NEW = df_NEW[cols_old]

        #df_NEW = fit_df(df_NEW, index_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

bill_cols = ["Site Code",\
             "Macro Region",\
             "Categorization by Transmission Sys",\
             "Categorization by Site Type",\
             "Core site type",\
             "Transmission Hub Site (YES/NO)",\
             "Transmission Hub Site (inc. with/without Shelters)",\
             "Transmission Site (inc. with/without Shelters)",\
             "Room configuration",\
             "Climate Control (YES/NO)",\
             "Power Supply ",\
             "Strategic Site (YES/NO)",\
             "Critical Site (YES/NO)",\
             "Is the Site a WIP site",\
             "	NOS shared site (Yes/No)",\
             "	MEO shared site (Yes/No)",\
             "BTS Sites (Yes/No)",\
             "Sites_As_Metered_Estimated",\
             "Strategic_Site_Bucket",\
             "Critical_Site_Beyond_10",\
             "First_Active_Sharing_Deployment_Type ",\
             "First_Active_Sharing_Start_Date",\
             "First_Active_Sharing_End_Date ",\
             "Subsequent_Sharing_Arrangement",\
             "Legacy_Site_Agreement_Terminated(Yes/NO)",\
             "Decommissioned Sites(True/false)"]

#'Billing Trigger Date',\
path = '/content/Vantage Towers_PT TowerDB Jun21_20210720.xlsx'
msa = '/content/TowerDB_Portugal_20210731.csv'
sheet = 'Final Delivery'
skiprows = 6
skipcolumns = 1
to_save = '/content/PT_TW'
old = 'TowerDB_Portugal_20210731.csv'
new = "TowerDB_Portugal_Jun21_20210720.xlsx"
dates_tw = ['infrastructure ready (existing)/ to be ready (new)', 'infrastructure to be dismantled by', 
            'date when vodafone active equipment is removed', 'infrastructure to be shared by']
#(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='', kind='tw', sheetname='', skipr=0, skipc=0)
find_diffs_between_files(msa, path, 'Site Code', bill_cols, to_save, old, new,type_file='mix',status_col='Status',kind='tw',\
                         dates=dates_tw, sheetname=sheet, skipr=6, skipc=1)


New Rows:     []
Dropped Rows: []

Done.



In [ ]:
def find_diffs_between_files_uis(path_OLD, path_NEW, index_col, \
                             path_save, old_name, new_name, type_file='mix', sheetname='', skipr=0, skipc=0):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df):

        df = df.dropna(subset=[index_col], axis=0)
        #df = df[uis_cols]
        #df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    #uis_cols = lower_str(uis_cols)
    #index_col = index_col.lower()
    # type_file = type_file.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD, sheet_name = sheetname,  skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = df_OLD.drop([0,0])
        df_OLD = fit_df(df_OLD)


        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname,  skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = df_NEW.drop([0,0]) 
        df_NEW = fit_df(df_NEW)


    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,uis_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,uis_cols)

    else:
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,uis_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,uis_cols)

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))
    print(sharedCols)
    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0,thresh=len(new_copy.columns[1:]))
    new_copy = new_copy.dropna(axis=1)

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - [{old_name}] vs [{new_name}].xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ1000', highlight)
    
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

col_uis = []
       
pathuis_n = ''
pathuis_o = '/content/UserInput_Portugal_20210731.xlsx'
sheetuis='SiteLevel'
skiprows=2
to_save = '/content/HU_UIS'

old_n = 'UIS - 20210630_true_up.xlsx'
new_n = 'UIS - 20210831.xlsx'

find_diffs_between_files_uis(pathuis_o, pathuis_n, 'Site_ID (Alphanumeric or Numeric)', \
                        to_save, old_n, new_n, type_file='excel', sheetname=sheetuis, skipr=2)

Entiry Check

Adding new columns for UIP

Change to openpyxl

In [ ]:
# Lendo o Ficheiro de input
path_uip='/content/UserInput_Portugal_20210731.xlsx'
sheet = 'SiteLevel'
skiprows = 6
skipcolumns = 1
site_level = pd.read_excel(path_uip,sheet, header=[0,1,2])
head_1 = pd.MultiIndex.from_product([[''],['Numeric (in months)'],['Delay in Site Modification Projects', 'Delay in BTS Projects']])
df_1 = pd.DataFrame(columns=head_1)

head_2 = pd.MultiIndex.from_product([[''],['Numeric (in €)'],['Excess of Upgrade Capital Expenditure over Threshold']])
df_2 = pd.DataFrame(columns=head_2)

site_level = pd.concat([site_level, df_1, df_2], axis=1)
site_level.head(1)

In [ ]:
cols_ordered = ["Site Code","Site Name","EVO Site ID","Macro Region","Region","Province","Municipality","Nr. Inhabitants","Address",\
                "Altitude","Latitude","Longitude","Ground Register","Categorization by inhabitants",\
                "Categorization by Transmission Sys","Categorization by Transmission Sys (sub-cluster)",\
                "Categorization by Site Type","Other internal categorisation I","Categorisation by connectivity",\
                "Technology VOD","Technology VOD: Auxiliar","Fibre / Microwave","Type of Structure","Tower Height (m)",\
                "Floor space","Floor space (availability)","Status",'Billing Trigger Date',\
                "Infrastructure to be dismantled by","Date when Vodafone active equipment is removed",\
                "Infrastructure to be shared by","Core site type","Transmission Hub Site (YES/NO)",\
                "Transmission Hub Site (inc. with/without Shelters)","Transmission Site (inc. with/without Shelters)",\
                "Room configuration","An Indoor + Outdoor Site","Climate Control (YES/NO)","Air Conditioning (YES/NO)",\
                "Free-Air Cooling (YES/NO)","Power Supply ","Active/Passive DAS","Strategic Site (YES/NO)","Critical Site (YES/NO)",\
                "	If a Non-Critical and non-Transmission Hub Site, Category 1 or Category 2","Is the Site a WIP site",\
                "Indicative completion date?","Capital expenditure incurred to the MSA Effective Date",\
                "Indicative capital expenditure required to complete the site build","Vertical Infrastructure ","Site Type",\
                "Technology_VOD","Fibre/Microwave","Power Supply","# Antennas","# Minilinks",\
                "	Details of Operator Equipment installed at the Site","Site Configuration","	NOS shared site (Yes/No)",\
                "	MEO shared site (Yes/No)",\
                "Details of any Active Sharing involving Operator applicable in respect of the Site (and, where applicable, the type of Active Sharing);",\
                "	Details of the access arrangements applicable at each Site (including any access restrictions and applicable public access requirements).",\
                "Sites under Hawkins arrangements (Yes/No)","Hawkins Sites (Yes/No)","NOS Crossed Site (Yes/No)","BTS Sites (Yes/No)",\
                "Energy Provider ","POD ID","Annual Energy Consumption (kWh)","Annual Energy cost (€)",\
                "Total Energy Cost (Energy provider + Landlord)","# Lease Contracts","Countepart","Total Annual Lease (€)",\
                "Current annual lease fees (€)","Current energy lease fees (€)","Current annual other fees (€)",\
                "Annual Lease fees + Annual Other fees","Sub-Lease","(Latest) residual duration until expiring date",\
                "Maturity Cluster - Expiring date","(Latest) residual duration until expiring date after renewal",\
                "Maturity Cluster - Expiring date after renewal","Type of lease contract","Reason for no lease fee",\
                "Comments to Lease Contracts","NOS (1/0)","Total Revenues NOS (€)","Annual Hosting fee NOS (€)",\
                "Annual Energy fee NOS (€)","Annual Maintenance fee NOS (€)","Annual Other Services fee NOS (€)",\
                "Tenancy duration until expiring date NOS","Maturity Clusters NOS","MEO (1/0)","Total Revenues MEO (€)",\
                "Annual Hosting fee MEO (€)","Annual Energy fee MEO (€)","Annual Maintenance fee MEO (€)",\
                "Annual Other Services fee MEO (€)","Tenancy duration until expiring date MEO","Maturity Clusters MEO",\
                "OTMOs (1/0)","Total Revenues OTMOs (€)","Annual Hosting fee OTMOs (€)","Annual Energy fee OTMOs (€)",\
                "Annual Maintenance fee OTMOs (€)","Annual Other Services fee OTMOs (€)","Tenancy duration until expiring date OTMOs",\
                "Maturity Clusters OTMOs","Total # of 3rd Party Tenants","Name of 3rd Party Tenants","Type of Sharing",\
                "Total Hosting Fee & Services from 3rd Party Tenants (€)","Annual Hosting fee from 3rd Party Tenants (€)",\
                "Annual Energy fee from 3rd Party Tenants (€)","Annual Maintenance fee from 3rd Party Tenants (€)",\
                "Other Services fee from 3rd Party Tenants (€)","Weighted Average tenancy duration until expiring date",\
                "Total # of Tenants","Macro Cluster Tenancy","Comments to Tenant Agreements","Macro Cluster Type of contract",\
                "Easement (Servitù di passaggio)","Turistic Sites","Sites_As_Metered_Estimated","Strategic_Site_Bucket",\
                "Critical_Site_Beyond_10","First_Active_Sharing_Deployment_Type ","First_Active_Sharing_Start_Date",\
                "First_Active_Sharing_End_Date ","Subsequent_Sharing_Arrangement","Legacy_Site_Agreement_Terminated(Yes/NO)",\
                "Decommissioned Sites(True/false)"]

In [14]:
# Lendo o Ficheiro de input
path ='/content/Vantage Towers_PT TowerDB Jun21_20210720.xlsx'
sheetname = 'Final Delivery'
skiprows = 6
skipcolumns = 1

towerdb = read_files(path, sheetname, skiprows, skipcolumns, "Site Code")
towerdb.rename(columns={'Billing Trigger Stop Date from May21 onwards': 'date_of_equipment_removal'}, inplace=True)
towerdb.columns = lower_str(list(towerdb.columns))
#towerdb = towerdb[lower_str(cols_ordered)]

"""First Step Check All columns received"""
path_msa = '/content/TowerDB_Portugal_20210731.csv'
df,cols_msa = csv_files(path_msa)
df_msa = pd.read_csv(path_msa, names=cols_msa, encoding='latin')
df_msa.columns = lower_str(list(df_msa.columns))
#df_msa = df_msa[lower_str(cols_ordered)]

# pegar as colunas de billing
bill_cols = ['Site Code','Macro Region','Categorization by Transmission Sys','Billing Trigger Date',\
             "Infrastructure to be dismantled by",'Date when Vodafone active equipment is removed','Core site type','Transmission Hub Site (YES/NO)',\
             'Transmission Hub Site (inc. with/without Shelters)','Transmission Site (inc. with/without Shelters)',\
             'Room configuration','Climate Control (YES/NO)','Power Supply ','Strategic Site (YES/NO)','Critical Site (YES/NO)',\
             'Is the Site a WIP site','	NOS shared site (Yes/No)','	MEO shared site (Yes/No)','BTS Sites (Yes/No)',\
             'Total Annual Lease (€)','Annual Lease fees + Annual Other fees','Sites_As_Metered_Estimated','Strategic_Site_Bucket',\
             'Critical_Site_Beyond_10','First_Active_Sharing_Deployment_Type ','First_Active_Sharing_Start_Date',\
             'First_Active_Sharing_End_Date ','Subsequent_Sharing_Arrangement','Legacy_Site_Agreement_Terminated(Yes/NO)',\
             'Decommissioned Sites(True/false)']

df_col_missing = check_columns_received(towerdb, list(df_msa.columns))
df_col_missing
# date_of_equipment_removal e infrastructure ready (existing)/ to be ready... não serão mais usadas
#infrastructure ready (existing)/ to be ready... passou a ser o billing trigger date
towerdb = towerdb[list(df_msa.columns)]

for i in ['province', 'municipality', 'address', 'site name']:
    towerdb[i] = towerdb[i].apply(unidecode)

ints = ['floor space', 'tower height (m)', 'floor space (availability)', 'annual energy consumption (kwh)' ]
for i in ints:
    towerdb[i] = towerdb[i].fillna(0)
    towerdb[i] = list(map(int, towerdb[i]))
    
dates_tw = ['infrastructure ready (existing)/ to be ready (new)', 'infrastructure to be dismantled by', 'date when vodafone active equipment is removed',
            'infrastructure to be shared by', 'indicative completion date?', 'date_of_equipment_removal']
for i in dates_tw:
    towerdb[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in towerdb[i]]

sites = [i for i in towerdb[towerdb['climate control (yes/no)']=='Yes; Indoor Air Conditioning and Free Air cooling / Free cooling units']['site code']]
towerdb.loc[towerdb['site code'].isin(sites), 'climate control (yes/no)'] = 'Yes; Indoor Air Conditioning'

sites = [i for i in towerdb[towerdb['climate control (yes/no)']=='Yes; Indoor Free Air cooling / Free cooling units']['site code']]
towerdb.loc[towerdb['site code'].isin(sites), 'climate control (yes/no)'] = 'Yes; Indoor Air Conditioning'

towerdb = towerdb.fillna('')

towerdb.head(2)


,site code,site name,evo site id,macro region,region,province,municipality,nr. inhabitants,address,altitude,latitude,longitude,ground register,categorization by inhabitants,categorization by transmission sys,categorization by transmission sys (sub-cluster),categorization by site type,other internal categorisation i,categorisation by connectivity,technology vod,technology vod: auxiliar,fibre / microwave,type of structure,tower height (m),floor space,floor space (availability),status,infrastructure ready (existing)/ to be ready (new),infrastructure to be dismantled by,date when vodafone active equipment is removed,infrastructure to be shared by,core site type,transmission hub site (yes/no),transmission hub site (inc. with/without shelters),transmission site (inc. with/without shelters),room configuration,an indoor + outdoor site,climate control (yes/no),air conditioning (yes/no),free-air cooling (yes/no),...,total revenues meo (€),annual hosting fee meo (€),annual energy fee meo (€),annual maintenance fee meo (€),annual other services fee meo (€),tenancy duration until expiring date meo,maturity clusters meo,otmos (1/0),total revenues otmos (€),annual hosting fee otmos (€),annual energy fee otmos (€),annual maintenance fee otmos (€),annual other services fee otmos (€),tenancy duration until expiring date otmos,maturity clusters otmos,total # of 3rd party tenants,name of 3rd party tenants,type of sharing,total hosting fee & services from 3rd party tenants (€),annual hosting fee from 3rd party tenants (€),annual energy fee from 3rd party tenants (€),annual maintenance fee from 3rd party tenants (€),other services fee from 3rd party tenants (€),weighted average tenancy duration until expiring date,total # of tenants,macro cluster tenancy,comments to tenant agreements,macro cluster type of contract,easement (servitù di passaggio),turistic sites,sites_as_metered_estimated,strategic_site_bucket,critical_site_beyond_10,first_active_sharing_deployment_type,first_active_sharing_start_date,first_active_sharing_end_date,subsequent_sharing_arrangement,legacy_site_agreement_terminated(yes/no),decommissioned sites(true/false),date_of_equipment_removal
0,1,BRAGA,PT11000001,CONTINENT,NORTE,Braga,Braga,182679,"Travessa Escola, 4705-474 Esporões",518,41.516600,-8.395440,,Urban,Macro,Standard,GBT,,,2G; 3G; 4G,,Fibre,Lattice,51,240,223,In Service,01/01/1992,,,,Non Core,Yes,With shelters,Non Transmission Site,Indoor,NO,Yes; Indoor Air Conditioning,Yes,No,...,0.0,0.0,0,0,0,,,0,0.0,0.0,0,0,0,,,0,,,0.0,0.0,0.0,0,0,,1,VF Only,,Leased Area - VF Only,,,Estimated Model,Non Strategic,Within 10%,,,,,No,No,
1,2,RECEZINHOS,PT13000002,CONTINENT,NORTE,Porto,Penafiel,69772,"Cruzes -Croca - Penafiel, 4560-871 Siforno",411,41.218686,-8.229177,,Urban,Macro,Standard,GBT,,,2G; 3G; 4G,,Fibre,Monopole,26,80,71,In Service,09/02/1992,,,,Non Core,No,Non Transmission Hub Site,Non Transmission Site,Indoor,NO,No,No,No,...,0.0,0.0,0,0,0,,,0,0.0,0.0,0,0,0,,,0,,,0.0,0.0,0.0,0,0,,1,VF Only,,Leased Area - VF Only,,,Estimated Model,Non Strategic,Non Critical,,,,,No,No,


In [38]:
from unidecode import unidecode

for i in ['province', 'municipality', 'address', 'site name']:
    towerdb[i] = towerdb[i].apply(unidecode)
towerdb.to_csv('/content/TowerDB_Portugal_20210831.csv', index=False)

In [ ]:
"""Defining variables which is gonna be reusable in checks"""
tw_index = 'site code'
tw_doer = 'date_of_equipment_removal'
tw_dismentled = "infrastructure to be dismantled by"
tw_status = 'status'
tw_bts = 'bts sites (yes/no)'
tw_bill ='billing trigger date'
tw_wip = 'is the site a wip site'
tw_decom = 'decommissioned sites(true/false)'
tw_critical = 'critical site(yes/no)'
tw_beyond = 'critical_site_beyond_10'

msa_index ='site code'
msa_doer = 'date_of_equipment_removal'
msa_status = 'status'
msa_bts = 'bts sites (yes/no)'
msa_bill = 'billing trigger date'
msa_wip = 'is the site a wip site'
msa_decom = 'decommissioned sites(true/false)'
msa_critical = 'critical site(yes/no)'
msa_beyond = 'critical_site_beyond_10'

Condição extra para retornar no log
Only Portugal

Fazer o check da contagem de Hawkins Sites. Se 

In [ ]:
def check_hawkins(df, tw_index, tw_hawkins):
    cont = len(towerdb[~(towerdb[tw_hawkins]=='No') & ~(towerdb[tw_hawkins]=='')][tw_index])
    sites = towerdb[~(towerdb[tw_hawkins]=='No') & ~(towerdb[tw_hawkins]=='')][tw_index]
    if not cont == 480:
        #Retornar a coluna toda e informar que está com error pois é maior que o esperado
        df = df.loc[df[tw_index].isin(sites)]
        return df[[tw_index, tw_hawkins]]
    else:
        print('There are 480 sites.')

tw_haw = 'hawkins sites (yes/no)'
df_hawkins = check_hawkins(towerdb, tw_index, tw_haw)
df_hawkins
# Missing Hawking Sites

There are 480 sites.


Create CSV Renan

In [ ]:
#Columns to parser
#infra = towerdb['infrastructure to be shared by'].astype('datetime64')
dates=[tw_bill, 'infrastructure to be shared by']
date_parser(towerdb, dates, "%d/%m/%Y", 'no')

#Sempre fazer o replace depois de todas as modificações
towerdb = replace_values(towerdb, lower_str(cols_ordered))

towerdb.round(2).to_csv('/content/TowerDB_Portugal_20210731.csv', index=False)

CSV (In Month and True Up) Checks

In [ ]:
path_tw_in = '/content/TowerDB_Portugal_20210731.csv'
df_in, cols_in = csv_files(path_tw_in)
cols_in = lower_str(cols_in)
twdb_month = pd.read_csv(path_tw_in, header=0, names=cols_in, engine='python',encoding='latin1')
#twdb_month.head(1)

In [ ]:
cols_ordered = lower_str(cols_ordered)
towerdb = replace_values(towerdb, cols_ordered)


In [ ]:
"""You need to convert all values in cols for string format to check"""
# Columns to functions
dates_bill = [tw_index, tw_bill]
dates_doer = [tw_index, tw_doer]
#Columns to parser
bill=[tw_bill]
doer=[tw_doer]

#Columns to parser
dates=[tw_bill, tw_doer_vf, tw_doer_wh]
date_parser(actives, dates, "%d/%m/%Y")

actives_1 = towerdb[towerdb['on air / active']=='In Service']
no_actives_1 = towerdb[~(towerdb[tw_status]=='In Service')]

# date_parser(actives_1, bill, "%d/%m/%Y", 'no')  não precisar checkar o bill dos sites in service
date_parser(no_actives_1, doer, "%d/%m/%Y", 'mixed')


In [ ]:
towerdb.columns.to_list()

In [ ]:
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            #df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
"""Second Check Date Format dd-mm-YYYY"""
date_cols = ['Site Code', 'infrastructure ready (existing)/ to be ready (new)', 'Date when Vodafone active equipment is removed', \
             'First_Active_Sharing_Start_Date', 'First_Active_Sharing_End_Date ', 'Subsequent_Sharing_Arrangement']
date_cols = lower_str(date_cols)
dates = ['infrastructure ready (existing)/ to be ready (new)', 'Date when Vodafone active equipment is removed', \
             'First_Active_Sharing_Start_Date', 'First_Active_Sharing_End_Date ', 'Subsequent_Sharing_Arrangement']
dates = lower_str(dates)
actives = towerdb[towerdb[tw_status]=='In Service']
date_parser(actives, dates, "%d/%m/%Y", 'no')             
#Check dates of column 'Date_Of_Equipment_Removal' Pg 7
"""Alterar o tipo da data para string para checkar"""
dftw_dates_errors = check_date_columns(actives, tw_index, tw_status, date_cols,2)
dftw_dates_errors 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,site code,date when vodafone active equipment is removed,first_active_sharing_start_date,first_active_sharing_end_date,subsequent_sharing_arrangement,status
0,1,Blank Value,Blank Value,Blank Value,Blank Value,In Service
1,2,Blank Value,Blank Value,Blank Value,Blank Value,In Service
2,3,Blank Value,Blank Value,Blank Value,Blank Value,In Service
3,4,Blank Value,Blank Value,Blank Value,Blank Value,In Service
4,5,Blank Value,Blank Value,Blank Value,Blank Value,In Service
...,...,...,...,...,...,...
3545,63083,Blank Value,Blank Value,Blank Value,Blank Value,In Service
3546,63856,Blank Value,Blank Value,Blank Value,Blank Value,In Service
3547,67476,Blank Value,Blank Value,Blank Value,Blank Value,In Service
3548,67540,Blank Value,Blank Value,Blank Value,Blank Value,In Service


In [ ]:
picklist_cols = ['Site Code', 'Categorization by Transmission Sys', 'Core site type', 'Transmission Hub Site (YES/NO)',\
                 'Transmission Hub Site (inc. with/without Shelters)','Transmission Site (inc. with/without Shelters)',\
                 'Room configuration', 'Climate Control (YES/NO)', 'Power Supply ', 'Strategic Site (YES/NO)',\
                 'Critical Site (YES/NO)', 'Is the Site a WIP site', '	NOS shared site (Yes/No)',\
                 '	MEO shared site (Yes/No)', 'BTS Sites (Yes/No)', 'Sites_As_Metered_Estimated', 'Strategic_Site_Bucket', \
                 'Critical_Site_Beyond_10', 'Legacy_Site_Agreement_Terminated(Yes/NO)', 'Decommissioned Sites(True/false)']

picklists = {
    'categorization by transmission sys': ['Macro','Macro+DAS', 'Public DAS','Repeater','Transmission'],
    'core site type': ['Non Core', 'Case A', 'Case B'],
    'transmission hub site (yes/no)': ['Yes','No'], 
    'transmission hub site (inc. with/without shelters)': ['Non Transmission Hub Site', 'With shelters', 'Without shelters'],
    'transmission site (inc. with/without shelters)': ['Non Transmission Site', 'Without shelters'],
    'room configuration': ['Outdoor', 'Indoor'],
    'climate control (yes/no)': ['Yes; Indoor Air Conditioning','', 'No'], 
    'power supply ': ['AC','DC'],
    'strategic site (yes/no)': ['Yes','No'],
    'critical site (yes/no)': ['Yes','No'],
    'is the site a wip site': ['Yes','No'],
    '	nos shared site (yes/no)': ['Yes','No'],
    '	meo shared site (yes/no)': ['Yes','No'],
    'bts sites (yes/no)': ['Yes','No'],
    'sites_as_metered_estimated': ['Estimated Model','Metered Model'],
    'strategic_site_bucket': ['Yes - 0-5%','Non Strategic'],
    'critical_site_beyond_10': ['Beyond 10%','Within 10%','Non Critical'],
    'legacy_site_agreement_terminated(yes/no)': ['Yes','No'],
    'decommissioned sites(true/false)': ['Yes','No']}

picklist_cols = lower_str(picklist_cols)

df_picklist= check_picklist(towerdb,tw_index,tw_status, picklist_cols, picklists)

#df_picklist_errors = check_picklist_3(tw ,twdb_index, picklists)
#Tem errors

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,site code,status,room configuration,climate control (yes/no),power supply
0,4,In Service,Ok!,Incorret picklist value: Yes; Indoor Air Condi...,Ok!
1,24,In Service,Ok!,Incorret picklist value: Yes; Indoor Air Condi...,Ok!
2,33,In Service,Ok!,Incorret picklist value: Yes; Indoor Air Condi...,Ok!
3,41,In Service,Ok!,Incorret picklist value: Yes; Indoor Air Condi...,Ok!
4,45,In Service,Ok!,Incorret picklist value: Yes; Indoor Air Condi...,Ok!
...,...,...,...,...,...
286,57558,WIP,Blank Value,Ok!,Blank Value
287,57716,Pipeline,Blank Value,Ok!,Blank Value
288,63858,Pipeline,Blank Value,Ok!,Blank Value
289,68058,Ordered,Blank Value,Ok!,Ok!


In [ ]:
def check_on_air_sites(df, df_index, df_cols):

    df_check = df[df_cols]
    df_check['sites'] = df_check[df_index]
    df_check = df_check.set_index('sites')
    #df_check = replace_values(df_check, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df_check[df_index]:
        for column in df_cols: 
            if df_check.loc[site,df_index] not in df_errors.index:
                df_errors.loc[site,df_index] = df_check.loc[site,df_index]
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
            else:
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    df_errors = df_errors[df_errors.iloc[:,1:]]
    df_errors = df_errors.dropna(how='all', axis=1).fillna('Ok')       
    #df = df_errors.dropna(how='all', axis=1)   
    return df_errors
# On the air sites which have blank values
on_air_columns = ['Site Code', 'Categorization by Transmission Sys','Categorization by Site Type',\
                  'Sites_As_Metered_Estimated', 'Infrastructure ready (existing)/ to be ready (new)', \
                  'Climate Control (YES/NO)', 'Strategic Site (YES/NO)', 'Critical Site (YES/NO)',\
                  'Is the Site a WIP site', 'Power Supply ', 'Strategic_Site_Bucket', \
                  'Critical_Site_Beyond_10','Legacy_Site_Agreement_Terminated(Yes/NO)', \
                  'Decommissioned Sites(True/false)']
on_air_columns = lower_str(on_air_columns)
#tw_on  = replace_values(towerdb, on_air_columns, 0)
df_on_air = check_on_air_sites(actives, tw_index, on_air_columns)
df_on_air
#tem error

In [ ]:
"""New Sites Check"""
# Read msa files to match site code information from towerdb file

# Read UIS
uip = pd.read_excel('/content/UserInput_Portugal_20210831.xlsx', sheet_name='SiteLevel', usecols=[0,1,2], skiprows=2).fillna('')
uip_col = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip.columns = lower_str(uip_col)
msa_sites = [i for i in df_msa[msa_index]]
tow_sites = [str(i) for i in towerdb[tw_index]]
uip_sites = [str(i) for i in uip['site_id']]

# return 2 df, first are the new sites with no BTS flags marked
# Second is the sites that are demerged dates than current date
#check_new_sites(towerdb, msa_sites, tow_sites, uip_sites, country)


In [ ]:
""" BTS sites"""
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    
    #list of new sites out of UIP sheet
    bts_out_uis = [i for i in df_towerdb[(df_towerdb[bts_col]=='Yes')&(df_towerdb[status_col]=='In Service')][tw_index] if i not in uip_list]
    bts_out_uis = pd.DataFrame(bts_out_uis, columns=['Bts_Sites_Out_UIS_File'])
    bts_out_uis = pd.merge(bts_out_uis, filtered, how='left', left_on=['Bts_Sites_Out_UIS_File'], right_on=tw_index)
    bts_out_uis = bts_out_uis[['Bts_Sites_Out_UIS_File', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    
    #if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
    return new_sites, bts_out_uis, df_site_bts[[tw_index,status_col, bts_col, bill_col]]


new_sites, bts_out_uis, df_site_bts = check_new_sites(towerdb,tw_index,tw_bts,tw_bill,tw_status, msa_sites, tow_sites, uip_sites)
#No one errors

In [ ]:
""" Wip Sites Check"""
def check_wip(df_tw,tw_index, wip_tw, tw_bts, tw_status, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[wip_tw]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, tw_status,wip_tw, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    return wip_out_tw_list, tw_wip_site_bts_flagged
wip_out_tw_list, df_wip_and_bts_flagged= check_wip(towerdb,tw_index, tw_wip,tw_bts, tw_status, df_msa,msa_index, msa_wip)
wip_out_tw_list
#Tem erro no df_wip_and_bts_flagged

,site code,status,is the site a wip site,bts sites (yes/no)
2774,8485,In Service,Yes,Yes
3445,52256,In Service,Yes,Yes
3513,55181,In Service,Yes,Yes
3514,55258,In Service,Yes,Yes
3516,55476,In Service,Yes,Yes
3523,63002,In Service,Yes,Yes
3524,63003,In Service,Yes,Yes
3525,63004,In Service,Yes,Yes
3526,63005,In Service,Yes,Yes
3527,63006,In Service,Yes,Yes


In [ ]:
"""Check Bts Sites"""
def check_bts(df_tw, bts_tw_columns, tw_index,  tw_status, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in bts_msa:
                if i not in tw_bts_sites:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = [str(i) for i in df_msa[df_msa[bts_msa_column]=='Yes'][msa_index]]

    tw_bts_sites = [str(i) for i in df_tw[df_tw[bts_tw_columns]=='Yes'][tw_index]]


    #return of datas
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    filter = df_tw[[tw_index, tw_status, bts_tw_columns]]
    filter[tw_index] = filter[tw_index].astype(str)
    df = pd.DataFrame(bts_out_tw, columns=['MSA BTS Sites not BTS in month'])
    df = pd.merge(df, filter, how='inner', left_on='MSA BTS Sites not BTS in month', right_on=tw_index)
    return df
#check_bts(df_tw, bts_tw_column, tw_index, df_msa, bts_msa_column, msa_index)
df_bts_out_tw = check_bts(towerdb,tw_bts,tw_index, tw_status, df_msa,msa_bts,msa_index)
df_bts_out_tw
#retorna os sites novos, ou não estavam na msa anteriormente

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,MSA BTS Sites not BTS in month,site code,status,bts sites (yes/no)
0,50252,50252,WIP,No
1,52091,52091,In Service,No
2,57558,57558,WIP,No
3,57716,57716,Pipeline,No
4,63858,63858,Pipeline,No
5,75881,75881,WIP,No


In [ ]:
"""Decomissioned Sites Check"""
#check_decommissioned(df,df_index, decom_col, doer_col)
df_decom = check_decommissioned(towerdb,tw_index, tw_decom, tw_dismentled)
df_decom
# No errors

,site code,decommissioned sites(true/false),infrastructure to be dismantled by


In [ ]:
"""DOER Check"""
#check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col)
df_doer = check_tw_bill_doer(towerdb,tw_index, tw_dismentled, tw_status, '', 'doer')
df_doer
# Tem Error

,site code,status,infrastructure to be dismantled by


In [ ]:
"""M-o-M check"""
df_mom = check_mom_bts(towerdb, tw_index, tw_bts, df_msa, msa_index, msa_bts)
df_mom
# No one error

,site code,bts sites (yes/no)


In [ ]:
""" Check UIS and Towerdb matchs"""

def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
    filtered[tw_index] = filtered[tw_index].astype(str) 
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [str(i) for i in filtered[filtered[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    print(uis_sites_not_in_towerdb)
    if uis_sites_not_in_towerdb:
        uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
        uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                        right_on=tw_index)
        uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    tw_decomiss = [str(i) for i in df_tw[df_tw[decom_col]=='Yes'][tw_index]]
    tw_decomiss = [i for i in tw_decomiss if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]

    #Check BTS sites
    """"bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    #uip_bts = [i for i in df_uip[df_uip['bts site applicable charge (annual)']!='']['site_id']]

    if len(uip_bts)>1:
        bts_sites_out_uip = []
        #if not set(bts_sites).intersection(uip_sites):
        bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='']['site_id']]
    bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
    print(len(uip_critical))
    critical = []
    if len(uip_critical) > 0:
        #if set(uip_critical).intersection(bts_tw_critical):
        critical = [i for i in uip_critical if i not in bts_tw_critical]
        critical = pd.DataFrame(critical, columns=['UIS Sites(critical beyond 10%) different in TowerDB'])
        critical = pd.merge(critical, filtered, how='left', left_on='UIS Sites(critical beyond 10%) different in TowerDB',\
                                    right_on=tw_index)
        critical = critical[['UIS Sites(critical beyond 10%) different in TowerDB', status_tw_col]]"""
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip

uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip = check_uip_tw(towerdb,tw_index,tw_status,tw_decom,tw_bts, tw_critical, uip, uip_sites)
# NO one errors

In [ ]:
in_service_not_in_uis

,TowerDB Sites out of UIS In Month!,status


In [ ]:
def check_diffs_v2(path_current, path_last, type_file='Excel', sheet='Commercial'):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)
    type_file = type_file.lower()
    if type_file =='excel':
        _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')

        _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

        df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
        df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

        #df_final.style.apply(highlight_diff, axis=None)
        if not df_final.empty:
            return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
        else:
            print('\nNo differences Founded!\n')
            
#cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type','Input_Value_actual', 'Input_Value_before' ,'Description/Instruction', 'Frequency of Update']
df_com = check_diffs_v2( '/content/UserInput_Portugal_20210831.xlsx', '/content/UserInput_Portugal_20210731.xlsx')


In [ ]:
df_list = [ dftw_dates_errors, df_picklist,df_wip_and_bts_flagged, df_com]
sheetnames = ['In service sites Dates Errors','Picklist Invalid Value', 'BTS and WIP columns Flagged', 'Commercial Values Differences']

path = '/content/towerdb_pt_errors.xlsx'
general_log_erros(df_list, sheetnames, path)

UIS Comparison(Old vs New)

In [ ]:
uis_new = '/content/UserInput_Portugal_20210831.xlsx'
uis_old = '/content/UserInput_Portugal_20210731.xlsx'
sheet = 'SiteLevel'
uis_index = 'Site_ID (Numeric)'
to_uis = '/content/PT_UIS_'
old_uis = 'UIS_20210731.xlsx'
new_uis = 'UIS_20210831.xlsx'
bill = []
find_diffs_between_files(uis_old, uis_new, uis_index, bill, to_uis, old_uis,new_uis,'excel',status_col='',\
                         kind='',kind_col='', sheetname=sheet, skipr=2, skipc=0)


New Rows:     ['4350']
Dropped Rows: []

Done.



TA Input Checks

In [28]:
# Lendo o Ficheiro de input
path='/content/Vantage Towers_PT TowerDB Jun21_20210720.xlsx'
#test = '/content/teste_dates.xlsx'
sheet_ta = 'Tenant Agreements'
skiprows_ta = 13
skipcol = 1
ta = read_files(path, sheet_ta, skiprows_ta, skipcol, "VF Site code")     
ta_output_cols = ["VF Site code","VF Site name","Other operator Site code","Other operator Site name",\
                    "OWNER","TENANT","Tenant classification","Type of sharing",'Annual Hosting Fee (in €)',\
                    "M1","M2","M3","M4","M5","M6","M7","M8","M9","M10","M11","M12","Annual Energy Fee (€)",\
                    "Annual Maintenance Fee (€)","Annual Other Services Fee (€)","Total Annual Fee (€)","Authorisation Date",\
                    "Fee Starting Date","Fee Expiring Date","Renewal Option","Expiring date after renewal","Termination Date",\
                    "Terms of payments","Payment type","Index","Indexation Driver","Percentage","VAT Subject","Percentage (VAT)",\
                    "Inputs from filling responsible","Tenancy duration until expiring date (in years)","Sharing In Scope","CONTINENT"]

print(len(ta_output_cols))
ta['Tenancy duration until expiring date (in years)'] = ['' for i in range(ta.shape[0])]
ta["CONTINENT"] = ['' for i in range(ta.shape[0])]
ta = ta[ta_output_cols]

dates_tw = ['Authorisation Date', 'Fee Starting Date', 'Fee Expiring Date']
for i in dates_tw:
    ta[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in ta[i]]

ta = ta.fillna('')

ta.to_csv('/content/TA_Input_Portugal_20210831.csv', index=False)
ta.head(2)
  

42


,VF Site code,VF Site name,Other operator Site code,Other operator Site name,OWNER,TENANT,Tenant classification,Type of sharing,Annual Hosting Fee (in €),M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12,Annual Energy Fee (€),Annual Maintenance Fee (€),Annual Other Services Fee (€),Total Annual Fee (€),Authorisation Date,Fee Starting Date,Fee Expiring Date,Renewal Option,Expiring date after renewal,Termination Date,Terms of payments,Payment type,Index,Indexation Driver,Percentage,VAT Subject,Percentage (VAT),Inputs from filling responsible,Tenancy duration until expiring date (in years),Sharing In Scope,CONTINENT
0,21,AVEIRO,99BL043,AVEIRO ESTE,VF,MEO,MNO,TOWER Sharing,2693.52,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,0.0,0,0,2693.52,01/03/1999,01/03/1999,,NO,,,EoP Deferred,Monthly,,,,YES,23%,,,Yes,
1,26,SAO MIGUEL DE MACHEDE,01AL016,S MIGUEL MACHEDE,VF,MEO,MNO,TOWER Sharing,2693.52,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,224.46,0.0,0,0,2693.52,01/06/2001,01/06/2001,,NO,,,EoP Deferred,Monthly,,,,YES,23%,,,Yes,


In [27]:
ta["Percentage (VAT)"].value_counts()

23%    748
         3
Name: Percentage (VAT), dtype: int64

In [29]:
df_ta_dates = check_lc_ta_dates(ta,"VF Site code", 'Fee Starting Date',"Fee Expiring Date")
df_ta_dates
#No erros

Empty DataFrame
Columns: [VF Site code, Fee Starting Date, Fee Expiring Date]
Index: []

No Errors Founded!



Comparison between oldest file and newest file

In [ ]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        if kind == 'ta':
            df['sites'] = df[index_col].astype(str) + df[kind_col]
            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = df_OLD.loc[row,col]
                    value_NEW = df_NEW.loc[row,col]
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old,  engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW,header=0, names=cols_new).fillna('')
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')


# Lendo o Ficheiro de input
path='/content/Vantage Towers_PT TowerDB Jun21_20210720.xlsx'
#test = '/content/teste_dates.xlsx'
skiprows_ta = 13
skipcol = 1

path_old_ta = '/content/TA_Input_Portugal_20210731.csv'
sheet_ta = 'Tenant Agreements'

ta_save = '/content/PT_TA'
old_ta = 'TA_Input_Portugal_20210630.csv'
new_ta = "TA_Input_Portugal_20210731.csv"
ta_cols = ['VF Site code', 'TENANT', 'Tenant classification','Authorisation Date', 'Fee Starting Date',"Fee Expiring Date","Sharing In Scope"]

"""path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0)"""
find_diffs_between_files(path_old_ta, path, 'VF Site code', ta_cols, ta_save, old_ta, new_ta,'mix', '', 'ta', 'TENANT', 
                         sheetname=sheet_ta, skipr=13, skipc=1)



New Rows:     []
Dropped Rows: []

Done.

